# Chapter 5 Test of Linear Independence

Let us decide whether vectors $\mathbf{a}_1,\ldots, \mathbf{a}_k \in \mathbb{R}^n$ are linear independent. We will follow two ways: Solving non-invertible linear system and SVD.

In [1]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [3]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

Recall the codes from Ch3-3 Solving Non-invertible Linear Systems to solve $A\mathbf{x} = \mathbf{b}$ where $A$ may be non-invertible.

In [16]:
def solve_Ax_b(A, b):
    m = A.shape[0]
    n = A.shape[1]
    Aaug = np.hstack((A,-b))
    # LU-decomposition
    P, L, U = sp.linalg.lu(Aaug)
    print(U)

    # find pivot entries
    epsilon = 10**(-10)     # Setting epsilon for negligibles
    pivot = n+1
    rank_u = 0
    pivots = []
    while pivot >= 0:
        non_zero_indices = np.nonzero(U[rank_u,:]**2 > epsilon)
        if np.size(non_zero_indices) == 0 or pivot == non_zero_indices[0][0] or rank_u == n:
            pivot = -1
        else:
            pivot = non_zero_indices[0][0]
            pivots.append(pivot)
            rank_u += 1

    print('Rank = ', rank_u)
    print('Pivot variables = ', np.int64(pivots))

    if n in pivots:
        print("The linear system is infeasible since right hand b increases the rank.")
        return
    
    # find column indices of free variables
    free_variables = np.sort(list(set(range(Aaug.shape[1]))- set(pivots)))
    print("Free variables = ", free_variables)

    # Collect pivot columns and free variables
    U_pivot = U[:rank_u, pivots]
    print('pivot columns of U = ')
    print(U_pivot)
    U_free = U[:rank_u, free_variables]
    print('free variable columns of U = ')
    print(U_free)

    # Solve for the pivot part of a basis of the null space of A
    X_pivot = np.linalg.solve(U_pivot,-U_free)
    print('pivot part of X = ')
    print(X_pivot)

    # remove the right hand b from free variables
    free_variables = free_variables[:-1]

    # Combine pivot part and free variable part
    X = np.hstack((np.eye(n)[:,free_variables], np.zeros(n).reshape(-1,1)))
    X[pivots,:] = X_pivot
    print('X = ')
    print(X)

    return np.allclose(A @ X, np.hstack((np.zeros((m,n-rank_u)),b)))   

In addition, we may use **numpy.linalg.matrix_rank** to check whether the columns of a matrix are linearly independent. **numpy.linalg.matrix_rank** is known to utilize **numpy.linalg.svd**. Let us also try to implement **numpy.linalg.matrix_rank** through **numpy.linalg.svd**.

In [5]:
def matrix_rank_scratch(A: np.ndarray, tol: float = 1e-10) -> int:
    """
    Compute the rank of a matrix using the SVD method.
    """
    U, s, Vh = np.linalg.svd(A, full_matrices=False)
    rank = np.sum(s > tol)
    return rank

Let us check the independence of columns of a matrix $A$.

In [18]:
# Setting solvability
independent = False
if independent:
    print("The vectors are linearly independent.")
else:
    print("The vectors are linearly dependent.")
# Setting parameters
m=13
n=10
# generate an mxn matrix of rank at most k
A = np.random.randn(m, n)

if not independent:
    x = np.random.randn(n, 3)
    # horizontal stack
    A = np.hstack((A, A@x))

print('linear independence of A is ', np.linalg.matrix_rank(A) == A.shape[1])
print('linear independence of A is ', matrix_rank_scratch(A) == A.shape[1])

# augment the matrix A with a row of ones
Aaug = np.vstack((A, np.ones((1, A.shape[1]))))
baug = np.zeros((Aaug.shape[0], 1))
baug[-1] = 1

print(Aaug.shape, baug.shape)
    
print('linear independence of A is ', not solve_Ax_b(Aaug,baug))

The vectors are linearly dependent.
linear independence of A is  False
linear independence of A is  False
(14, 13) (14, 1)
[[ 1.7598 -0.6531  0.8887  0.2602  0.674  -2.0969  0.4982  2.093  -1.3789  1.6687  2.2496 -0.3788
  -5.4121 -0.    ]
 [ 0.      1.3711  0.495   0.8522  0.617   2.1916  0.7169 -0.1893  1.7836  0.0518 -0.2783  1.2152
   4.0754 -1.    ]
 [ 0.      0.      2.9184  0.7492 -1.2212  4.0872  0.6597  1.0134  2.2346  1.3649  6.0067  0.1859
  -0.6953 -0.6102]
 [ 0.      0.      0.      1.6971  1.8356  0.4129  0.4969  2.2707  1.4144  1.2758  1.8545 -0.4779
  -0.8499 -0.5995]
 [ 0.      0.      0.      0.     -2.397   0.3786  0.4382  0.926   0.5091  1.4724 -1.4435 -1.3205
  -5.7845  0.1485]
 [ 0.      0.      0.      0.      0.      4.5411 -1.4261 -1.1994  0.4865 -2.8555  3.0597 -2.6188
   6.8476  0.1221]
 [ 0.      0.      0.      0.      0.      0.      1.8363 -0.7962 -0.3393  1.6248  2.5698  3.5226
  -0.5466 -0.251 ]
 [ 0.      0.      0.      0.      0.      0.      0.     